In [5]:
import pandas as pd

train = pd.read_csv('test.csv', encoding='utf-8', nrows=200)
train.head()

,sample_id,genre_id,ts_listen,media_id,album_id,context_type,release_date,platform_name,platform_family,media_duration,listen_type,user_gender,user_id,artist_id,user_age
0,0,50,1478104371,683078,82356,1,20021008,0,0,542,1,0,17698,2076,30
1,1,2744,1479317140,876497,99692,1,19851231,0,0,307,1,0,10525,26,28
2,2,2744,1479546361,876497,99692,1,19851231,0,0,307,1,0,8716,26,27
3,3,2744,1478457729,876500,99692,1,19851231,2,1,265,1,0,5443,26,30
4,4,2744,1480448560,876504,99692,1,19851231,2,1,356,1,0,7600,26,29


In [29]:
import json, requests
import librosa
from array import array
from six.moves.urllib.request import urlretrieve

url = 'https://api.deezer.com/track/'
mfcc = []
mel = []
rank = []
bpm = []
gain = []
filename = 'test.mp3'
for _,el in enumerate(train.media_id.unique()) :
    url1 = url + str(el)
    resp = requests.get(url=url1)
    data = json.loads(resp.text)
    preview = data['preview']
    if preview == '':
        preview = data['alternative']['preview']
        rank.append(data['alternative']['rank'])
        bpm.append(data['alternative']['bpm'])
        gain.append(data['alternative']['gain'])
    else:
        rank.append(data['rank'])
        bpm.append(data['bpm'])
        gain.append(data['gain'])
            
    urlretrieve(preview, filename=filename)
    y, sr = librosa.load(filename)
    mel.append(librosa.feature.melspectrogram(y=y, sr=sr,n_mels=10))
    mfcc.append(librosa.feature.mfcc(y=y, sr=sr,n_mfcc=10))
    


In [30]:
mel

[array([[  3.83247796e+00,   3.57138864e+01,   7.87129166e+01, ...,
           3.04198861e+00,   2.40419619e+00,   1.36120050e+00],
        [  1.14066972e+00,   1.30832770e+01,   2.79164510e+01, ...,
           3.58977421e-01,   3.48017428e-01,   2.13834306e-01],
        [  1.59378102e-01,   1.24040523e+00,   2.69287323e+00, ...,
           7.06763321e-02,   9.38339228e-02,   6.94042212e-02],
        ..., 
        [  9.12958763e-03,   1.07601094e-01,   2.54994822e-01, ...,
           1.52962623e-03,   1.52706162e-03,   8.36305645e-04],
        [  2.38963785e-03,   2.20475413e-02,   4.57070474e-02, ...,
           3.62603393e-04,   2.92724381e-04,   1.53933964e-04],
        [  5.84505202e-04,   3.90012211e-03,   8.76962279e-03, ...,
           1.71194020e-04,   9.35546991e-05,   4.34008882e-05]]),
 array([[  5.74270694e-10,   1.51031442e-02,   2.56745698e-01, ...,
           2.59806898e+00,   2.26625799e+00,   1.54477399e+00],
        [  2.72987603e-11,   2.54060792e-04,   1.21809081e-0